Based on https://www.kaggle.com/kredy10/simple-lstm-for-text-classification and https://www.kaggle.com/mlwhiz/learning-text-classification-textcnn

# Import the necessary libraries

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Reshape, Conv2D, MaxPool2D, Concatenate, Flatten
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
%matplotlib inline

In [ ]:
SEED = 42

In [ ]:
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

### Load the data into Pandas dataframe

In [ ]:
!wget https://github.com/Pittawat2542/krathu-500/raw/main/labeled/comments-small-labeled.csv

In [ ]:
df = pd.read_csv('comments-small-labeled.csv')
df.head()

* Create input and output vectors.
* Process the labels.

In [ ]:
X = df.text.astype(str)
Y = df.class_label.astype(str)
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = to_categorical(Y)

Split into training and test data.

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.15, random_state = SEED)

### Process the data
* Tokenize the data and convert the text to sequences.
* Add padding to ensure that all the sequences have the same shape.
* There are many ways of taking the *max_len* and here an arbitrary length of 150 is chosen.

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

### CNN
Define the CNN structure.

In [ ]:
def CNN():
    filter_sizes = [1,2,3,5]
    num_filters = 36
    embed_size = 50

    inputs = Input(name='inputs',shape=[max_len])
    x = Embedding(max_words, embed_size, input_length=max_len)(inputs)
    x = Reshape((max_len, embed_size, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size), kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(max_len - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    output = Dense(len(le.classes_), activation="sigmoid")(z)

    model = Model(inputs=inputs, outputs=output)
    return model

Call the function and compile the model.

In [ ]:
model = CNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Fit on the training data.

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

The model performs well on the validation set and this configuration is chosen as the final model.

Process the test set data.

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

Evaluate the model on the test set.

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))